Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [1]:
import pandas as pd
import numpy as np

# Loading the bank marketing csv into a pandas dataframe
bank_marketing = pd.read_csv('bank_marketing.csv')
print(bank_marketing.head(n=5))

   client_id  age        job  marital    education credit_default housing  \
0          0   56  housemaid  married     basic.4y             no      no   
1          1   57   services  married  high.school        unknown      no   
2          2   37   services  married  high.school             no     yes   
3          3   40     admin.  married     basic.6y             no      no   
4          4   56   services  married  high.school             no      no   

  loan    contact month  ...  campaign  pdays  previous     poutcome  \
0   no  telephone   may  ...         1    999         0  nonexistent   
1   no  telephone   may  ...         1    999         0  nonexistent   
2   no  telephone   may  ...         1    999         0  nonexistent   
3   no  telephone   may  ...         1    999         0  nonexistent   
4  yes  telephone   may  ...         1    999         0  nonexistent   

   emp_var_rate cons_price_idx  cons_conf_idx  euribor3m  nr_employed   y  
0           1.1         93.9

In [2]:
#Splitting the bank marketing dataframe into three dataframes using the information provided

original_client_columns = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']

original_campaign_columns = ['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'day', 'month']

original_economics_columns = ['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']

client = bank_marketing[original_client_columns]
campaign = bank_marketing[original_campaign_columns]
economics = bank_marketing[original_economics_columns]

print(client.head(5))
print(campaign.head(5))
print(economics.head(5))

   client_id  age        job  marital    education credit_default housing loan
0          0   56  housemaid  married     basic.4y             no      no   no
1          1   57   services  married  high.school        unknown      no   no
2          2   37   services  married  high.school             no     yes   no
3          3   40     admin.  married     basic.6y             no      no   no
4          4   56   services  married  high.school             no      no  yes
   client_id  campaign  duration  pdays  previous     poutcome   y  day month
0          0         1       261    999         0  nonexistent  no   13   may
1          1         1       149    999         0  nonexistent  no   19   may
2          2         1       226    999         0  nonexistent  no   23   may
3          3         1       151    999         0  nonexistent  no   27   may
4          4         1       307    999         0  nonexistent  no    3   may
   client_id  emp_var_rate  cons_price_idx  euribor3m  nr_

**Renaming specific columns according to the criteria below:**

"client_id" to "id" in **client** (leave as-is in the other subsets);

"duration" to "contact_duration", 
"previous" to "previous_campaign_contacts",
"y" to "campaign_outcome", 
"poutcome" to "previous_outcome", and 
"campaign" to "number_contacts" in **campaign**;

"euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in **economics**.

In [3]:
# Renaming columns in three dataframes

client.rename(columns = {'client_id':'id'}, inplace=True)

campaign.rename(columns = {'duration':'contract_duration',                     'previous':'previous_campaign_contracts', 'y':'campaign_outcome',
'poutcome':'previous_outcome', 'campaign':'number_contracts'}, inplace=True)

economics.rename(columns = {'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'}, inplace=True)

# Printing a list of columns for wach dataframe
print(client.keys())
print(campaign.keys())
print(economics.keys())

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['client_id', 'number_contracts', 'contract_duration', 'pdays',
       'previous_campaign_contracts', 'previous_outcome', 'campaign_outcome',
       'day', 'month'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2447106245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client.rename(columns = {'client_id':'id'}, inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2447106245.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign.rename(columns = {'duration':'contract_duration',                     'previous':'previous_campaign_contracts', 'y':'campaign_outcome',
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2447106245.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

**The data cleaning process**

To clean the "education" column in the **client** dataframe, by changing "." to "_", and "unknown" to NumPy's null values. 

To clean the "job column" in the **client** dataframe. The objective is to remove periods from the column.

To clean the "previous_outcome" and "campaign_outcome" columns in the **campaign** dataframe.
Converting "success" and "failure" in the "previous_outcome" column and "yes" and "no" in the "campaign_outcome" to binary (1 or 0).
Changing "nonexistent" to NumPy's null values in "previous_outcome".

Adding a column called "campaign_id" in *campaign*, with all rows having a value of 1.

Creating a datetime column called "last_contract_date" in the *campaign* dataframe, with the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.

Removing any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table descriptions provided.

In [4]:
# Cleaning the education column

client['education'] = client['education'].str.replace('.', '_', regex=False)
client['education'] = client['education'].replace('unknown', np.nan)

# Displaying the unique values in the education column to determine if the cleaning process was done successfully
client.education.unique()

C:\Users\USER\AppData\Local\Temp\ipykernel_8560\130836403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].str.replace('.', '_', regex=False)
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\130836403.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].replace('unknown', np.nan)


array(['basic_4y', 'high_school', 'basic_6y', 'basic_9y',
       'professional_course', nan, 'university_degree', 'illiterate'],
      dtype=object)

In [5]:
# Cleaning the job column

client['job'] = client['job'].str.replace('.', '', regex=False)

# Displaying the unique values in the job column to determine if the cleaning process was done successfully
client.job.unique()

C:\Users\USER\AppData\Local\Temp\ipykernel_8560\3673292071.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['job'] = client['job'].str.replace('.', '', regex=False)


array(['housemaid', 'services', 'admin', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [6]:
# Cleaning the previous_outcome and campaign_outcome columns

# Define a mapping for "previous_outcome"
prev_outcome_mapping = {'success': 1, 'failure': 0, 'nonexistent': np.nan}

# Define a mapping for "campaign_outcome"
camp_outcome_mapping = {'yes': 1, 'no': 0}

# Convert values in "previous_outcome" and "campaign_outcome" using the mapping
campaign['previous_outcome'] = campaign['previous_outcome'].map(prev_outcome_mapping)

campaign['campaign_outcome'] = campaign['campaign_outcome'].map(camp_outcome_mapping)

# Displaying the unique values in the previous_outcome and campaign_outcome columns to determine if the cleaning process was done successfully
print(campaign.previous_outcome.unique())
print(campaign.campaign_outcome.unique())

[nan  0.  1.]
[0 1]


C:\Users\USER\AppData\Local\Temp\ipykernel_8560\1656872217.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['previous_outcome'] = campaign['previous_outcome'].map(prev_outcome_mapping)
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\1656872217.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['campaign_outcome'] = campaign['campaign_outcome'].map(camp_outcome_mapping)


Note that the dtype of the previous outcome column is float64 due to the np.nan method, whereas the dtype of the campaign_outcome column is int64.

In [7]:
# Adding the campaign_id column in campaign, with all rows having a value of 1
campaign['campaign_id'] = 1

campaign.head(5)

C:\Users\USER\AppData\Local\Temp\ipykernel_8560\998125946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['campaign_id'] = 1


,client_id,number_contracts,contract_duration,pdays,previous_campaign_contracts,previous_outcome,campaign_outcome,day,month,campaign_id
0,0,1,261,999,0,NaN,0,13,may,1
1,1,1,149,999,0,NaN,0,19,may,1
2,2,1,226,999,0,NaN,0,23,may,1
3,3,1,151,999,0,NaN,0,27,may,1
4,4,1,307,999,0,NaN,0,3,may,1


In [8]:
# Create the "year" column
campaign['year'] = 2022

# Create the "last_contact_date" column
campaign['last_contact_date'] = pd.to_datetime(campaign['year'].astype(str) + '-' + campaign['month'].astype(str) + '-' + campaign['day'].astype(str), format='%Y-%b-%d')

# Print the DataFrame with the new "last_contact_date" column
campaign.head(5)

C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2169199984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['year'] = 2022
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2169199984.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['last_contact_date'] = pd.to_datetime(campaign['year'].astype(str) + '-' + campaign['month'].astype(str) + '-' + campaign['day'].astype(str), format='%Y-%b-%d')


,client_id,number_contracts,contract_duration,pdays,previous_campaign_contracts,previous_outcome,campaign_outcome,day,month,campaign_id,year,last_contact_date
0,0,1,261,999,0,NaN,0,13,may,1,2022,2022-05-13
1,1,1,149,999,0,NaN,0,19,may,1,2022,2022-05-19
2,2,1,226,999,0,NaN,0,23,may,1,2022,2022-05-23
3,3,1,151,999,0,NaN,0,27,may,1,2022,2022-05-27
4,4,1,307,999,0,NaN,0,3,may,1,2022,2022-05-03


In [9]:
# Removing the day, month and year columns, to avoid redundancy
campaign = campaign.drop(columns=['day', 'month', 'year'])

campaign.head(5)

,client_id,number_contracts,contract_duration,pdays,previous_campaign_contracts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13
1,1,1,149,999,0,NaN,0,1,2022-05-19
2,2,1,226,999,0,NaN,0,1,2022-05-23
3,3,1,151,999,0,NaN,0,1,2022-05-27
4,4,1,307,999,0,NaN,0,1,2022-05-03


The next step is to save the three dataframes to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.

In [10]:
# Saving the three dataframes as csv files

client.to_csv('client.csv', index=False)

campaign.to_csv('campaign.csv', index=False)

economics.to_csv('economics.csv', index=False)

After saving the dataframes to csv files, the last process is to design the bank marketing database.

Writing SQL code as a string to create three database tables, client, campaign, and economics.

In [11]:
# Creating python variables to create a table called client using values from client.csv 

client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);

\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE,
);

\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);

\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

**All done!!**